# Prescriptor submission


In [ ]:
import pandas as pd

## Inputs

### Past interventions
Like for predict.py, prescribe.py expects the list of past interventions for a list of countries. For example:

In [ ]:
EXAMPLE_IP_FILE = "covid_xprize/validation/data/2020-09-30_historical_ip.csv"
ip_df = pd.read_csv(EXAMPLE_IP_FILE,
                    parse_dates=['Date'],
                    dtype={"RegionName": str},
                    encoding="ISO-8859-1")
ip_df.tail()

### Intervention costs
Intervention plan costs can differ across regions. For example, closing public transportation may be
much costlier in London than it is in Los Angeles. Such preferences are expressed as weights
associated with each intervention plan dimension, given to the prescriptor as input for each region.

For example:

In [ ]:
EXAMPLE_COSTS_FILE = "covid_xprize/validation/data/uniform_random_costs.csv"
costs_df = pd.read_csv(EXAMPLE_COSTS_FILE,
                       dtype={"RegionName": str},
                       encoding="ISO-8859-1")
costs_df.head()

## Expected output
A call to `prescribe.py` should produce this kind of file:

In [ ]:
EXAMPLE_OUTPUT_FILE = "2020-08-01_2020-08-04_prescriptions_example.csv"
prediction_output_df = pd.read_csv(EXAMPLE_OUTPUT_FILE,
                                   parse_dates=['Date'],
                                   encoding="ISO-8859-1")
prediction_output_df.head()

# Prescribe

In [ ]:
start_date = "2020-08-01"
end_date = "2020-08-04"
ip_file = "covid_xprize/validation/data/2020-09-30_historical_ip.csv"
costs_file = "covid_xprize/validation/data/uniform_random_costs.csv"
output_file = "prescriptions/2020-08-01_2020-08-04.csv"

In [ ]:
!python prescribe.py -s {start_date} -e {end_date} -ip {ip_file} -c {costs_file} -o {output_file}

In [ ]:
!head {output_file}

# Validate

In [ ]:
from covid_xprize.validation.prescriptor_validation import validate_submission

In [ ]:
errors = validate_submission(start_date, end_date, ip_file, output_file)
if errors:
    for error in errors:
        print(error)
else:
    print("All good!")

# Evaluate

## Load generated prescriptions

In [ ]:
presc_df = pd.read_csv(output_file)

In [ ]:
prescription_indexes = list(presc_df.PrescriptionIndex.unique())
print(f"Loaded {len(prescription_indexes)} prescriptions:")
prescription_indexes

In [ ]:
# Look at the first prescription plan
idx = 0
idx_df = presc_df[presc_df['PrescriptionIndex'] == idx]
idx_df = idx_df.drop(columns='PrescriptionIndex') # Predictor doesn't need this
idx_df.head()

In [ ]:
# Save it to a file
presc_x_file = "prescriptions/2020-08-01_2020-08-04_0.csv"
idx_df.to_csv(presc_x_file)

## Compute prescription plan number of cases
Run the prescription plan through the predictor to get its predicted number of daily cases

In [ ]:
# Run the standard predictor
pred_x_file = "predictions/2020-08-01_2020-08-04_0.csv"
!python covid_xprize/standard_predictor/predict.py -s {start_date} -e {end_date} -ip {presc_x_file} -o {pred_x_file}



In [ ]:
# Load the predictions
preds_x_df = pd.read_csv(pred_x_file)
preds_x_df.head()

## Compute prescription plan stringency

In [ ]:
# Load IP cost weights
cost_df = pd.read_csv(costs_file)
cost_df.head()

In [ ]:
NPI_COLS = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']

In [ ]:
def weight_prescriptions_by_cost(pres_df, cost_df):
    """
    Weight prescriptions by their costs.
    """
    weighted_df = pres_df.merge(cost_df, how='outer', on=['CountryName', 'RegionName'], suffixes=('_pres', '_cost'))
    for npi_col in NPI_COLS:
        weighted_df[npi_col] = weighted_df[npi_col + '_pres'] * weighted_df[npi_col + '_cost']
    return weighted_df

In [ ]:
# Apply weights to prescriptions
weighted_x_df = weight_prescriptions_by_cost(idx_df, cost_df)
weighted_x_df.head()

In [ ]:
weighted_x_df['Stringency'] = weighted_x_df[NPI_COLS].sum(axis=1)

In [ ]:
weighted_x_df.head()

In [ ]:
mean_new_cases = preds_x_df.PredictedDailyNewCases.mean()
mean_stringency = weighted_x_df.Stringency.mean()
print(f"Mean predicted daily new cases: {mean_new_cases}")
print(f"Mean stringency: {mean_stringency}")

# Robojudge
Time to compare these prescriptions to others: have a look at the [prescriptor robojudge notebook](prescriptor_robojudge.ipynb)